# Topics
1. ML and MLLib libraries 
  - Data frame vs RDD based , latter getting deprecated
2. Matrix support 
  - Sparse and Dense vectors and matrices 
3. Working with libSVM kind of data. 
4. Feature Transformers  
5. Feature Extractors   
6. Feature Selectors 
7. Model selection and Tuning 
8. Pipelines
    

#### 2. Linear Algebra Module 
- Dense vectors, matrices   
  - Dense and Sparse are interconvertible, Dense rep. is essentially same as numpy array 
- Sparse vectors, matrices 
  - Can take in scipy.sparse type matrices ( check how to create)

In [ ]:
import numpy as np
import scipy as sp
import sklearn as sk

from pyspark.ml.linalg import SparseMatrix, DenseMatrix, Vectors

x = Vectors.dense(np.arange(1,20,1))
x
print(type(x))

x.dot(x)

In [ ]:
from pyspark.ml.linalg import SparseVector
a = SparseVector(4, [1, 3], [3.0, 4.0])
a

# conversion of dense matri to sparse or numpy array
x = DenseMatrix(5,6, range(30))
x

x.toSparse()

x.toArray()

# sparse matrix types in scipy - two of ones that allow inverse calculation
from scipy.sparse import csc_matrix, csr_matrix 

A = csr_matrix([[1, 2, 0], [0, 0, 3], [4, 0, 5]])
A

print(A)

B = csc_matrix([[1, 2, 0], [0, 0, 3], [4, 0, 5]])
B

print(B)

from pyspark.ml.linalg import Vectors
denseVec = Vectors.dense(1.0, 2.0, 3.0)
size = 3
idx = [1, 2] # locations of non-zero elements in vector
values = [2.0, 3.0]
sparseVec = Vectors.sparse(size, idx, values)


#### 3. Working with libSVM kind of data.  
- libsvm is a popular data format for large scale ML esp, used in popular SVM libraries LIBSVM and LIBLINEAR. 
- Format is - 
  - label index1:value1 index2:value2 ...  
  - index is 1 based, post reading made to 0 based in spark

In [ ]:
libsvm = spark.read.format('libsvm').load('/user/sumad/Data/sample_libsvm_data.txt')
libsvm.show(5)
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+

#### 4. Feature transformers 
- Ref : https://spark.apache.org/docs/2.3.0/ml-features.html

#### 4.1 Categorical feature encoding 

4.1.1 String Indexer  
- Creates a numeric index on a categorical column, essentially assigning frequenct based index to each 
  category  
- When fit on a new data set, three ways to handle new categories encountered : error, remove records, assign 
  a new index to all new cats.

In [ ]:
s_ind = StringIndexer(inputCol= 'category', outputCol = 'catIndex',
handleInvalid='error', stringOrderType='frequencyDesc')
df_indexed = s_ind.fit(df).transform(df)
df_indexed.show()

+---+--------+--------+
| id|category|catIndex|
+---+--------+--------+
|  0|       a|     0.0|
|  1|       b|     2.0|
|  2|       c|     1.0|
|  3|       a|     0.0|
|  4|       a|     0.0|
|  5|       c|     1.0|
+---+--------+--------+

4.1.2 One hot encoding 
- Unlike sklearn's one hot encoder, as default, it uses a feature vector of length n-1 to represent all categories of a column . dropLast = True ensure n-1 feature vec length
- Treatment of new category is as in StringIndexer. 
- The ouput representation is a sparse vector  
- **When dropLast = True, any new category is thus assigned all 0 vector**  
- OneHotEncoderEstimator can transform multiple columns, returning an one-hot-encoded output vector column for each input column. It is common to merge these vectors into a single feature vector using VectorAssembler.

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator
df = spark.createDataFrame([
    (0.0, 1.0),
    (1.0, 9.0),
    (2.0, 1.0),
    (3.0, 2.0),
    (0.0, 1.0),
    (2.0, 4.0)
], ["categoryIndex1", "categoryIndex2"])

oh_enc = OneHotEncoderEstimator(inputCols= ["categoryIndex1", "categoryIndex2"],
                      outputCols= ["oh_categoryIndex1", "oh_categoryIndex2"],
                      handleInvalid='error',dropLast=True)
df_enc = oh_enc.fit(df).transform(df)

+--------------+--------------+-----------------+-----------------+
|categoryIndex1|categoryIndex2|oh_categoryIndex1|oh_categoryIndex2|
+--------------+--------------+-----------------+-----------------+
|           0.0|           1.0|    (3,[0],[1.0])|    (9,[1],[1.0])|
|           1.0|           9.0|    (3,[1],[1.0])|        (9,[],[])|
|           2.0|           1.0|    (3,[2],[1.0])|    (9,[1],[1.0])|
|           3.0|           2.0|        (3,[],[])|    (9,[2],[1.0])|
|           0.0|           1.0|    (3,[0],[1.0])|    (9,[1],[1.0])|
|           2.0|           4.0|    (3,[2],[1.0])|    (9,[4],[1.0])|
+--------------+--------------+-----------------+-----------------+

In [ ]:
IndextoString
Response Encoding

Inreraction ( gives a cartesian product, how is it used)

4. 2 VectorIndexer (Improves performance, automates cat. identification of columns)  
  - **Operates on columns of types vectors, which means columns were concatenated to form a vector columns**  
  - selects categorical vectors from continious based on set threshold on levels 
  - Indexing is performed UNLIKE StringIndexer, it is based on ordered values, not ordered frequencies
  - ***Not reliable, gives weird results below**

In [ ]:
>>> df = spark.createDataFrame([(Vectors.dense([-1.0, 0.0, 2.0]),),(Vectors.dense([0.0, 1.0, 1.0]),), (Vectors.dense([-1.0, 1.0, 2.0]),)], ["a"])
>>> indexer = VectorIndexer(maxCategories=3, inputCol="a", outputCol="indexed")
>>> model = indexer.fit(df)
>>> df.show()
+--------------+
|             a|
+--------------+
|[-1.0,0.0,2.0]|
| [0.0,1.0,1.0]|
|[-1.0,1.0,2.0]|
+--------------+

>>> model.categoryMaps
{0: {0.0: 0, -1.0: 1}, 1: {0.0: 0, 1.0: 1}, 2: {1.0: 0, 2.0: 1}}

Vector Assembler 
- Bsically concatenates columns of types numeric, vector, boolean to create a single column of vector type 

In [ ]:
>>> from pyspark.ml.feature import VectorAssembler
>>> 
>>> dataset = spark.createDataFrame(
...     [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
...     ["id", "hour", "mobile", "userFeatures", "clicked"])
>>> 
>>> assembler = VectorAssembler(
...     inputCols=["hour", "mobile", "userFeatures"],
...     outputCol="features")
>>> 
>>> output = assembler.transform(dataset)
>>> dataset.show()                                                              
+---+----+------+--------------+-------+
| id|hour|mobile|  userFeatures|clicked|
+---+----+------+--------------+-------+
|  0|  18|   1.0|[0.0,10.0,0.5]|    1.0|
+---+----+------+--------------+-------+

>>> output.show()
+---+----+------+--------------+-------+--------------------+
| id|hour|mobile|  userFeatures|clicked|            features|
+---+----+------+--------------+-------+--------------------+
|  0|  18|   1.0|[0.0,10.0,0.5]|    1.0|[18.0,1.0,0.0,10....|
+---+----+------+--------------+-------+--------------------+

In [ ]:
4.2 Continuous Feature Transformers 
- Scalers 
- Bucketizer, like cut in pandas 
- QuantileDiscretizer

Normalizer 
- transforms a dataset of vector rows, applying a p norm to each row  
- **It is a transformer, not an estimator; unlike other scalers like Standard Scaler which need to be fit on data**

In [ ]:
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
>>> l1NormData = normalizer.transform(dataFrame)
>>> print("Normalized using L^1 norm")
Normalized using L^1 norm
>>> l1NormData.show()
+---+--------------+------------------+                                         
| id|      features|      normFeatures|
+---+--------------+------------------+
|  0|[1.0,0.5,-1.0]|    [0.4,0.2,-0.4]|
|  1| [2.0,1.0,1.0]|   [0.5,0.25,0.25]|
|  2|[4.0,10.0,2.0]|[0.25,0.625,0.125]|
+---+--------------+------------------+

Standard Scaler 
- Acts on a vector column, but unlike Normalizer, normalizes across column elements , using std and mean (optional) 
- If std is 0, returns 0 

In [ ]:
>>> from pyspark.ml.feature import StandardScaler
>>> scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
...                         withStd=True, withMean=False)
>>> scalerModel = scaler.fit(dataFrame)
>>>                                                                             
>>> # Normalize each feature to have unit standard deviation.
... scaledData = scalerModel.transform(dataFrame)

>>> scaledData.show(3, False)
+---+--------------+------------------------------------------------------------+
|id |features      |scaledFeatures                                              |
+---+--------------+------------------------------------------------------------+
|0  |[1.0,0.5,-1.0]|[0.6546536707079772,0.09352195295828244,-0.6546536707079771]|
|1  |[2.0,1.0,1.0] |[1.3093073414159544,0.1870439059165649,0.6546536707079771]  |
|2  |[4.0,10.0,2.0]|[2.618614682831909,1.870439059165649,1.3093073414159542]    |
+---+--------------+------------------------------------------------------------+

MinMaxScalrer
- Transforms within a given range 
MaxAbsScaler  
- Transforms between -1 and 1, does not destroy sparsity of data 

4.3 PolynomialExpansion

In [ ]:
>>> from pyspark.ml.feature import PolynomialExpansion
>>> from pyspark.ml.linalg import Vectors
>>> 
>>> df = spark.createDataFrame([
...     (Vectors.dense([2.0, 1.0]),),
...     (Vectors.dense([0.0, 0.0]),),
...     (Vectors.dense([3.0, -1.0]),)
... ], ["features"])
>>> 
>>> polyExpansion = PolynomialExpansion(degree=3, inputCol="features", outputCol="polyFeatures")
>>> polyDF = polyExpansion.transform(df)
>>> 
>>> polyDF.show(truncate=False)
+----------+------------------------------------------+                         
|features  |polyFeatures                              |
+----------+------------------------------------------+
|[2.0,1.0] |[2.0,4.0,8.0,1.0,2.0,4.0,1.0,2.0,1.0]     |
|[0.0,0.0] |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]     |
|[3.0,-1.0]|[3.0,9.0,27.0,-1.0,-3.0,-9.0,1.0,3.0,-1.0]|
+----------+------------------------------------------+

#### Pipelines 
- Like sci-lit learn, concept of transformer, estimator 
- model, once fit goes from becoming an estimaor to a transformer 
- Every estimatir has a fit() method to learn params from data, and transformer had transform() 
- predict() is replaced by transform()
- Important attributes of estimator 
  - explainParams() and extractParamMap() post fitting

In [ ]:
# Normal Example 
>>> # Prepare training data from a list of (label, features) tuples.
... training = spark.createDataFrame([
...     (1.0, Vectors.dense([0.0, 1.1, 0.1])),
...     (0.0, Vectors.dense([2.0, 1.0, -1.0])),
...     (0.0, Vectors.dense([2.0, 1.3, 1.0])),
...     (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

>>> training.show(10)
+-----+--------------+                                                          
|label|      features|
+-----+--------------+
|  1.0| [0.0,1.1,0.1]|
|  0.0|[2.0,1.0,-1.0]|
|  0.0| [2.0,1.3,1.0]|
|  1.0|[0.0,1.2,-0.5]|
+-----+--------------+

# parameters can be specified using .paramname on an estimator
lr = LogisticRegression(maxIter=10, regParam=0.01)
paramMap = {lr.regParam: 0.1, lr.threshold: 0.55}
model1 = lr.fit(training, paramMap)

prediction = model2.transform(test)

>>> prediction.show()
+-----+--------------+--------------------+--------------------+----------+     
|label|      features|       rawPrediction|       myProbability|prediction|
+-----+--------------+--------------------+--------------------+----------+
|  1.0|[-1.0,1.5,1.3]|[-2.8119038522838...|[0.05668429360932...|       1.0|
|  0.0|[3.0,2.0,-0.1]|[2.48711787928029...|[0.92323378664212...|       0.0|
|  1.0|[0.0,2.2,-1.5]|[-2.0865940788370...|[0.11040665017928...|       1.0|
+-----+--------------+--------------------+--------------------+----------+

In [ ]:
#### Pipeline Example 
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer


>>> tokenizer = Tokenizer(inputCol="text", outputCol="words")
>>> hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
>>> lr = LogisticRegression(maxIter=10, regParam=0.001)
>>> pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
>>> model = pipeline.fit(training)
prediction = model.transform(test)

#### ML models work on features combined together in a single column which of data type vector 

#### Model Tuning 
- Cross Validation strategy 
  - CrossValidator 
  - TrainValidationSplit.  
- Building a parameter grid 
  - ParamGridBuilder 
  - **By default evaluation is done sequentially, however, parallilim can be set as > 1 in CrossValidator or TrainValidationSplit, which i think creates separate partitions, ideally  < 10**
- Using Eval metrics available from Evaluator Module, each is a class
  - RegressionEvaluator 
  - BinaryClassificationEvaluator
  - MulticlassClassificationEvaluator 
  - **Default evaluator in estimator can be over ridden by setting setMetricName** 

In [ ]:
+---+----------------+-----+                                                    
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
|  4|     b spark who|  1.0|
|  5|         g d a y|  0.0|
|  6|       spark fly|  1.0|
|  7|   was mapreduce|  0.0|
|  8| e spark program|  1.0|
|  9|         a e c l|  0.0|
| 10|   spark compile|  1.0|
| 11| hadoop software|  0.0|
+---+----------------+-----+

>>> from pyspark.ml import Pipeline
>>> from pyspark.ml.classification import LogisticRegression
>>> from pyspark.ml.evaluation import BinaryClassificationEvaluator
>>> from pyspark.ml.feature import HashingTF, Tokenizer
>>> from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=2, parallelism = 2)
cvModel = crossval.fit(training)
prediction = cvModel.transform(test)
+---+---------------+------------------+--------------------+--------------------+--------------------+----------+
| id|           text|             words|            features|       rawPrediction|         probability|prediction|
+---+---------------+------------------+--------------------+--------------------+--------------------+----------+
|  4|    spark i j k|  [spark, i, j, k]|(1000,[105,149,32...|[-1.0143531895130...|[0.26612878920913...|       1.0|
|  5|          l m n|         [l, m, n]|(1000,[6,638,655]...|[2.45505377427970...|[0.92093023893998...|       0.0|
|  6|mapreduce spark|[mapreduce, spark]|(1000,[105,953],[...|[-0.2292614916964...|[0.44293435984699...|       1.0|
|  7|  apache hadoop|  [apache, hadoop]|(1000,[181,495],[...|[1.80181132002375...|[0.85836928288627...|       0.0|
+---+---------------+------------------+--------------------+--------------------+--------------------+----------+